# データセット構成状態モニタリング

ここでは、作成いただいたデータセット（実験リポジトリ）の構成状態をモニタリングします。以下のセルを実行することで、それぞれのデータセットがその管理品質上、適合的であるかどうかを確認できます。

## ◇◆◇開発メモ◇◆◇

### 処理の流れ

- DMPの情報として、`dirStructure`が定義されている想定。何種類かのスキームから選んでもらうようなイメージでいる。
    - 現在、設定しているのは「RCOS_with_code」「RCOS_only_data」「YODA」の３種類。
- `dirStructure`の値を取得。暫定的に、実験名とスキーム名の辞書型としている。
- 実験名のリストとスキーム名のリストとをそれぞれ作成し、モニタリング処理を回す。
    - 現状、スキームごとに望ましいフォルダ構成を定義しており、それぞれのパスが実際に存在するかを確認し、その結果を出力するような実装になっている。


In [ ]:
import os
import json
from typing import List


def get_ideal_paths(scheme_name: str, path_to_dataset: str) -> List[str]:
    # 存在することが望ましいディレクトリ・ファイルのリスト
    ideal_paths_RCOS_with_code = [
        path_to_dataset + '/source',
        path_to_dataset + '/input_data',
        path_to_dataset + '/output_data',
        path_to_dataset + '/Dockerfile',
        path_to_dataset + '/.gitattributes',
        path_to_dataset + '/.gitignore',
        path_to_dataset + '/meta_data.json',
        path_to_dataset + '/experiment.ipynb'
    ]

    ideal_paths_RCOS_only_data = [
        path_to_dataset + '/input_data',
        path_to_dataset + '/output_data',
        path_to_dataset + '/Dockerfile',
        path_to_dataset + '/.gitattributes',
        path_to_dataset + '/.gitignore',
        path_to_dataset + '/meta_data.json',
        path_to_dataset + '/experiment.ipynb'
    ]

    ideal_paths_YODA = [
        path_to_dataset + '/ci',
        path_to_dataset + '/code',
        path_to_dataset + '/code/tests',
        path_to_dataset + '/docs',
        path_to_dataset + '/docs/build',
        path_to_dataset + '/docs/source',
        path_to_dataset + '/envs',
        path_to_dataset + '/inputs',
        path_to_dataset + '/inputs/data',
        path_to_dataset + '/outputs',
        path_to_dataset + '/CHANGELOG.md',
        path_to_dataset + '/HOWTO.md',
        path_to_dataset + '/README.md',
    ]

    return eval('ideal_paths_' + scheme_name)

###########################################################################################################


def monitor_dir_structure(scheme_name: str, paths_to_dataset: List[str]) -> None:
    ideal_paths: List[str] = []
    dataset_name: str = ''

    for dataset in paths_to_dataset:
        ideal_paths = get_ideal_paths(scheme_name, dataset)
        dataset_name = os.path.basename(dataset)

        print('================================ Monitoring Dataset Structure ================================')
        print('Dataset name: ' + dataset_name)
        print(' STATUS    PATH')

        for path in ideal_paths:
            if os.path.exists(path):
                print('  OK    ' + path)
            else:
                print('  NG    ' + path)
                print('        [WARN] Your dataset structure is deprecated.')

        print('\nMonitered by "' + scheme_name + '" scheme.\n')

###########################################################################################################

# スクリプトファイルの拡張子リストを定義（あるいは読み出し）
# something cool...


def main() -> None:
    path_monitoring_params = '/home/jovyan/WORKFLOW/FLOW/param_files/monitoring_params.json'

    # データセットのリストを作る
    base_path = '/home/jovyan/experiments/'
    files = os.listdir(base_path)
    paths_to_dataset = [base_path + f for f in files if os.path.isdir(os.path.join(base_path, f))]

    scheme_name = ''
    with open(path_monitoring_params, "r") as f:
        dmp = json.load(f)
        scheme_name = dmp["dirStructure"]

    monitor_dir_structure(scheme_name, paths_to_dataset)


if __name__ == '__main__':
    main()
